## RATON BASIN FORMATION TOPS

##### Some functions adapted from https://sainosmichelle.github.io/elements.html and https://github.com/Rocks-n-Code/PythonCourse

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import warnings
import time
import requests
warnings.filterwarnings('ignore')

In [3]:
df_CO = pd.read_csv('../data/inputs/COGCC wells.csv') # downloaded from COGCC all Colorado wells

In [4]:
#Extract wells only Raton Basin 
twp_list = ['31S', '32S', '33S', '34S', '35S']
range_list = ['61W', '62W', '63W', '64W', '65W', '66W', '67W', '68W']
temp = df_CO[df_CO['Township'].isin(twp_list)] 
df_Raton_CO = temp[temp['Range'].isin(range_list)] 

In [5]:
df_Raton_CO[['Facil_Type',  'Facil_Stat', 'Facil_Id']]

Facil_Type Facil_Stat  Facil_Id
37999       WELL         DM    217167
38000       WELL         DA    217951
38001       WELL         PA    217169
38004       WELL         DA    217172
38005       WELL         DA    217173
...          ...        ...       ...
41972       WELL         AL    437234
41981       WELL         AL    292200
41983       WELL         DA    217952
41984       WELL         pa    217953
41985       WELL         TA    217954

[3881 rows x 3 columns]

In [6]:
status_list = ['PR', 'PA', 'TA']# 'SU', 'SI']
df_Raton_CO_act = df_Raton_CO[df_Raton_CO['Facil_Stat'].isin(status_list)]
len(df_Raton_CO_act)

3077

In [48]:
# taken from Matthew Bauer PG
def top_parse(text):
    tops = text.split('<!-- LOOP FOR EACH Formation WITHIN WELLBORE -->')[1].split('<!-- END Formation LOOP -->')[0]
    tops = tops.replace('\r','').replace('\n','').replace('\t','').strip()
    tops = tops.split('<tr>')
    tops = [x.strip() for x in tops if len(x) > 0]
    
    formations = {}
    for top in tops:
        cols = top.split('<td')
        cols = [x.strip() for x in cols if len(x) > 0]    
        formation = cols[0].split('color="Navy">')[1].split(' ')[0]
        depth = eval(cols[1].split('<font size="2">')[1].split('</font>')[0])
        formations[formation] = depth
    return formations

In [49]:
#FOR loop to extract all formation tops available in COGCC Raton Basin

wells_list = df_Raton_CO_act.api10.values#[0:49]
#wells_list = [507106224, 507106360, 507106650, 507106111, 507160012, 507160013]
topsDF = pd.DataFrame()
baseURL = 'https://cogcc.state.co.us/cogis/FacilityDetail.asp?facid='
tailURL = '&type=WELL'

for idx in range(len(wells_list)):
    url = baseURL + str(wells_list[idx])[1:] + tailURL
    r = requests.get(url)
    #print('URL:', url)
    try:
        formations = top_parse(r.text)
        formations['API'] = wells_list[idx] 
        topsDF = topsDF.append(formations,ignore_index=True)
        time.sleep(3) #Wait 3 sec.
    except:
        pass

In [50]:
topsDF.head(15)

API  DAKOTA  GREENHORN  MORRISON  NIOBRARA  TIMPAS  GRANEROS  \
0   507105004.0     NaN        NaN       NaN       NaN     NaN       NaN   
1   507105010.0  4882.0     4693.0    5076.0    3736.0     NaN       NaN   
2   507105017.0     NaN        NaN       NaN       NaN     NaN       NaN   
3   507105027.0     NaN        NaN       NaN       NaN     NaN       NaN   
4   507105028.0     NaN        NaN       NaN       NaN     NaN       NaN   
5   507105030.0     NaN        NaN       NaN       NaN     NaN       NaN   
6   507105035.0     NaN        NaN       NaN       NaN     NaN       NaN   
7   507105047.0     NaN        NaN       NaN       NaN  1252.0       NaN   
8   507105049.0     NaN        NaN       NaN       NaN     NaN       NaN   
9   507105054.0  1885.0     1470.0       NaN       NaN  1100.0    1840.0   
10  507105060.0     NaN        NaN       NaN       NaN     NaN       NaN   
11  507105062.0     NaN        NaN       NaN       NaN     NaN       NaN   
12  507105069.0     NaN        NaN       NaN       NaN     NaN       NaN   
13  507106022.0     NaN        NaN       NaN       NaN     NaN       NaN   
14  507106023.0     NaN     1100.0    1370.0     609.0     NaN       NaN   

    PIERRE  APISHAPA  CODELL  ...  POISON  TRINIDAD  CARLILE  RATON-VERMEJO  \
0      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
1      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
2      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
3      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
4      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
5      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
6      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
7      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
8      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
9      NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
10     NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
11     NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
12     NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
13     NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   
14     NaN       NaN     NaN  ...     NaN       NaN      NaN            NaN   

    ENTRADA-DOCKUM  GLORIETA  PURGATOIRE  UNKNOWN  TORONTO  ROLLINS  
0              NaN       NaN         NaN      NaN      NaN      NaN  
1              NaN       NaN         NaN      NaN      NaN      NaN  
2              NaN       NaN         NaN      NaN      NaN      NaN  
3              NaN       NaN         NaN      NaN      NaN      NaN  
4              NaN       NaN         NaN      NaN      NaN      NaN  
5              NaN       NaN         NaN      NaN      NaN      NaN  
6              NaN       NaN         NaN      NaN      NaN      NaN  
7              NaN       NaN         NaN      NaN      NaN      NaN  
8              NaN       NaN         NaN      NaN      NaN      NaN  
9              NaN       NaN         NaN      NaN      NaN      NaN  
10             NaN       NaN         NaN      NaN      NaN      NaN  
11             NaN       NaN         NaN      NaN      NaN      NaN  
12             NaN       NaN         NaN      NaN      NaN      NaN  
13             NaN       NaN         NaN      NaN      NaN      NaN  
14             NaN       NaN         NaN      NaN      NaN      NaN  

[15 rows x 23 columns]

In [ ]:
topsDF.info()

In [51]:
topsDF.to_csv('../data/outputs/allRaton_tops.csv')